# 1. gecleante Netflix und gecleante Disney mergen

In [7]:
import pandas as pd

# Pfade zu den CSV-Dateien
netflix_cleaned_file = '../../1_Datenset/erstellte/cleaned/netflix_titles_cleaned.csv'
disney_cleaned_file = '../../1_Datenset/erstellte/cleaned/disney_plus_titles_cleaned.csv'

# Lade die Netflix CSV-Datei und füge eine Spalte für die Plattform hinzu
netflix_cleaned_df = pd.read_csv(netflix_cleaned_file)
netflix_cleaned_df['platform'] = 'Netflix'

# Lade die Disney+ CSV-Datei und füge eine Spalte für die Plattform hinzu
disney_cleaned_df = pd.read_csv(disney_cleaned_file)
disney_cleaned_df['platform'] = 'Disney+'

# Finde gemeinsame Titel
common_titles_cleaned = set(netflix_cleaned_df['title']).intersection(set(disney_cleaned_df['title']))

# Markiere gemeinsame Titel in beiden DataFrames
netflix_cleaned_df['platform'] = netflix_cleaned_df.apply(lambda row: 'Netflix, Disney+' if row['title'] in common_titles_cleaned else 'Netflix', axis=1)
disney_cleaned_df['platform'] = disney_cleaned_df.apply(lambda row: 'Netflix, Disney+' if row['title'] in common_titles_cleaned else 'Disney+', axis=1)

# Kombiniere die DataFrames
combined_cleaned_df = pd.concat([netflix_cleaned_df, disney_cleaned_df], ignore_index=True)

# Entferne doppelte Einträge
combined_cleaned_df = combined_cleaned_df.drop_duplicates(subset=['title'])

# Benenne die Spalte 'rating' in 'agerating' um
combined_cleaned_df.rename(columns={'rating': 'agerating'}, inplace=True)

# Gruppiere nach allen Spalten außer 'platform' und kombiniere die Plattformen
combined_cleaned_df = combined_cleaned_df.groupby(['show_id', 'type', 'title', 'director', 'cast', 'country', 'release_year', 'agerating', 'duration', 'listed_in', 'description'], as_index=False).agg({'platform': lambda x: ', '.join(sorted(set(x)))})

# Neuverteilung der show_id von oben nach unten in der Form 's1', 's2', ...
combined_cleaned_df['show_id'] = ['s' + str(i+1) for i in range(len(combined_cleaned_df))]

# Gib das kombinierte DataFrame aus
print(combined_cleaned_df.head())

# Speichere das kombinierte DataFrame in einer neuen CSV-Datei
combined_cleaned_df.to_csv('../../1_Datenset/erstellte/fertig/fertig.csv', index=False)


  show_id   type                                             title  \
0      s1  Movie                              Dick Johnson Is Dead   
1      s2  Movie  Duck the Halls: A Mickey Mouse Christmas Special   
2      s3  Movie             A Muppets Christmas: Letters To Santa   
3      s4  Movie                                      The Starling   
4      s5  Movie                       Confessions of a Shopaholic   

                            director  \
0                    Kirsten Johnson   
1  Alonso Ramirez Ramos, Dave Wasson   
2                   Kirk R. Thatcher   
3                     Theodore Melfi   
4                         P.J. Hogan   

                                                cast        country  \
0                                            unknown  United States   
1  Chris Diamantopoulos, Tony Anselmo, Tress MacN...        unknown   
2  Steve Whitmire, Dave Goelz, Bill Barretta, Eri...  United States   
3  Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...  

In [8]:
# Zeige die eindeutigen Werte in der Spalte 'platform'
unique_platforms = combined_cleaned_df['platform'].unique()
print(f"Einzigartige Werte in der Spalte 'platform': {unique_platforms}")

# Zähle die Anzahl der Einträge für jede Plattform
cleaned_netflix_count = combined_cleaned_df[combined_cleaned_df['platform'] == 'Netflix'].shape[0]
cleaned_disney_count = combined_cleaned_df[combined_cleaned_df['platform'] == 'Disney+'].shape[0]
cleaned_both_count = combined_cleaned_df[combined_cleaned_df['platform'] == 'Netflix, Disney+'].shape[0]

print(f"Anzahl der Netflix-Einträge: {cleaned_netflix_count}")
print(f"Anzahl der Disney+-Einträge: {cleaned_disney_count}")
print(f"Anzahl der Einträge auf beiden Plattformen: {cleaned_both_count}")


Einzigartige Werte in der Spalte 'platform': ['Netflix' 'Disney+' 'Netflix, Disney+']
Anzahl der Netflix-Einträge: 8733
Anzahl der Disney+-Einträge: 1400
Anzahl der Einträge auf beiden Plattformen: 43


## 2. merge movies dazu


In [77]:
movies_file = '../../1_Datenset/ursprüngliche/movies.csv'
movies_df = pd.read_csv(movies_file)

# Entferne die angegebenen Spalten
movies_df.drop(columns=['budget', 'original_language', 'id', 'revenue', 'spoken_languages', 'status', 'title', 'crew', 'tagline', 'homepage', 'keywords', 'popularity', 'production_companies'], inplace=True)

# Benenne die Spalten im movies DataFrame um
movies_df.rename(columns={
    'original_title': 'title',
    'genres': 'genre',
    'overview': 'description',
    'runtime': 'duration',
    'director': 'director',
    'cast': 'cast',
    'production_countries' : 'country',
}, inplace=True)

# Speichere das DataFrame mit den umbenannten Spalten zurück in die CSV-Datei
movies_df.to_csv('../../1_Datenset/erstellte/movies_renamed.csv', index=False)


# Berechne die minimalen, maximalen und durchschnittlichen Werte für 'vote_count' und 'vote_average'
min_vote_count = movies_df['vote_count'].min()
max_vote_count = movies_df['vote_count'].max()
avg_vote_count = movies_df['vote_count'].mean()

min_vote_average = movies_df['vote_average'].min()
max_vote_average = movies_df['vote_average'].max()
avg_vote_average = movies_df['vote_average'].mean()

print(f"Minimale Anzahl an Stimmen: {min_vote_count}")
print(f"Maximale Anzahl an Stimmen: {max_vote_count}")
print(f"Durchschnittliche Anzahl an Stimmen: {avg_vote_count:.2f}")

print(f"Minimale durchschnittliche Bewertung: {min_vote_average}")
print(f"Maximale durchschnittliche Bewertung: {max_vote_average}")
print(f"Durchschnittliche Bewertung: {avg_vote_average:.2f}")


Minimale Anzahl an Stimmen: 0
Maximale Anzahl an Stimmen: 13752
Durchschnittliche Anzahl an Stimmen: 690.22
Minimale durchschnittliche Bewertung: 0.0
Maximale durchschnittliche Bewertung: 10.0
Durchschnittliche Bewertung: 6.09


In [82]:

# Pfade zu den CSV-Dateien
merged_file = '../../1_Datenset/erstellte/merged_netflix_disney_titles.csv'
movies_file = '../../1_Datenset/erstellte/movies_renamed.csv'

# Lade die merged CSV-Datei
merged_df = pd.read_csv(merged_file)

# Lade die Movies CSV-Datei
movies_df = pd.read_csv(movies_file)

# Wähle nur die relevanten Spalten aus
movies_df = movies_df[['title', 'vote_count', 'vote_average']]

# Benenne die Spalten um
movies_df.rename(columns={'vote_count': 'votes', 'vote_average': 'score'}, inplace=True)

# Führe den Merge basierend auf den Titeln durch
merged_df = pd.merge(merged_df, movies_df, on='title', how='left')

# Gib das kombinierte DataFrame aus
print(merged_df.head())

# Speichere das kombinierte DataFrame in einer neuen CSV-Datei
merged_df.to_csv('../../1_Datenset/erstellte/merged_with_votes.csv', index=False)

# Berechne die Anzahl der fehlenden Werte in den Spalten 'vote_count' und 'vote_average'
missing_vote_count = merged_df['votes'].isnull().sum()
missing_vote_average = merged_df['score'].isnull().sum()

print(f"Anzahl der fehlenden Werte in 'vote_count': {missing_vote_count}")
print(f"Anzahl der fehlenden Werte in 'vote_average': {missing_vote_average}")

# Berechne die Anzahl der vorhandenen Werte in den Spalten 'vote_count' und 'vote_average'
existing_vote_count = merged_df['votes'].notnull().sum()
existing_vote_average = merged_df['score'].notnull().sum()

print(f"Anzahl der vorhandenen Werte in 'vote_count': {existing_vote_count}")
print(f"Anzahl der vorhandenen Werte in 'vote_average': {existing_vote_average}")

  show_id   type                                             title  \
0      s1  Movie                              Dick Johnson Is Dead   
1      s1  Movie  Duck the Halls: A Mickey Mouse Christmas Special   
2     s10  Movie             A Muppets Christmas: Letters To Santa   
3     s10  Movie                                      The Starling   
4    s100  Movie                       Confessions of a Shopaholic   

                            director  \
0                    Kirsten Johnson   
1  Alonso Ramirez Ramos, Dave Wasson   
2                   Kirk R. Thatcher   
3                     Theodore Melfi   
4                         P.J. Hogan   

                                                cast        country  \
0                                            unknown  United States   
1  Chris Diamantopoulos, Tony Anselmo, Tress MacN...        unknown   
2  Steve Whitmire, Dave Goelz, Bill Barretta, Eri...  United States   
3  Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...  

## 3. Merge IMBD Movies von kaggle dazu

In [85]:
# Pfade zu den CSV-Dateien
kaggle_file = '../../1_Datenset/ursprüngliche/movies_kaggle.csv'
merged_file = '../../1_Datenset/erstellte/merged_with_votes.csv'

# Lade die Kaggle CSV-Datei
kaggle_df = pd.read_csv(kaggle_file)

# Lade die merged CSV-Datei
merged_df = pd.read_csv(merged_file)

# Wähle nur die relevanten Spalten aus
kaggle_df = kaggle_df[['title', 'score', 'votes']]

# Führe den Merge basierend auf den Titeln durch
merged_df = pd.merge(merged_df, kaggle_df, on='title', how='left', suffixes=('', '_kaggle'))

# Fülle die leeren Zellen in 'score' und 'votes' mit den Werten aus 'score_kaggle' und 'votes_kaggle'
merged_df['score'].fillna(merged_df['score_kaggle'], inplace=True)
merged_df['votes'].fillna(merged_df['votes_kaggle'], inplace=True)

# Entferne die zusätzlichen Spalten
merged_df.drop(columns=['score_kaggle', 'votes_kaggle'], inplace=True)

# Gib das kombinierte DataFrame aus
print(merged_df.head())

# Speichere das kombinierte DataFrame in einer neuen CSV-Datei
merged_df.to_csv('../../1_Datenset/erstellte/2_merged_with_votes.csv', index=False)

# Berechne die Anzahl der fehlenden Werte in den Spalten 'vote_count' und 'vote_average'
missing_vote_count_2 = merged_df['votes'].isnull().sum()
missing_vote_average_2 = merged_df['score'].isnull().sum()

print(f"Anzahl der fehlenden Werte in 'vote_count': {missing_vote_count_2}")
print(f"Anzahl der fehlenden Werte in 'vote_average': {missing_vote_average_2}")

# Berechne die Anzahl der vorhandenen Werte in den Spalten 'vote_count' und 'vote_average'
existing_vote_count_2 = merged_df['votes'].notnull().sum()
existing_vote_average_2 = merged_df['score'].notnull().sum()

print(f"Anzahl der vorhandenen Werte in 'vote_count': {existing_vote_count_2}")
print(f"Anzahl der vorhandenen Werte in 'vote_average': {existing_vote_average_2}")

  show_id   type                                             title  \
0      s1  Movie                              Dick Johnson Is Dead   
1      s1  Movie  Duck the Halls: A Mickey Mouse Christmas Special   
2     s10  Movie             A Muppets Christmas: Letters To Santa   
3     s10  Movie                                      The Starling   
4    s100  Movie                       Confessions of a Shopaholic   

                            director  \
0                    Kirsten Johnson   
1  Alonso Ramirez Ramos, Dave Wasson   
2                   Kirk R. Thatcher   
3                     Theodore Melfi   
4                         P.J. Hogan   

                                                cast        country  \
0                                            unknown  United States   
1  Chris Diamantopoulos, Tony Anselmo, Tress MacN...        unknown   
2  Steve Whitmire, Dave Goelz, Bill Barretta, Eri...  United States   
3  Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...  

/var/folders/q4/y1pgtglx2tl3lyq5msp5vs8m0000gp/T/ipykernel_37276/2743761577.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['score'].fillna(merged_df['score_kaggle'], inplace=True)
/var/folders/q4/y1pgtglx2tl3lyq5msp5vs8m0000gp/T/ipykernel_37276/2743761577.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are

## 4. netflix tv shows


In [86]:

# Pfade zu den CSV-Dateien
netflix_file = '../../1_Datenset/ursprüngliche/Netflix TV Shows and Movies.csv'
merged_file = '../../1_Datenset/erstellte/2_merged_with_votes.csv'

# Lade die Netflix CSV-Datei
netflix_df = pd.read_csv(netflix_file)

# Lade die merged CSV-Datei
merged_df = pd.read_csv(merged_file)

# Wähle nur die relevanten Spalten aus
netflix_df = netflix_df[['title', 'score', 'votes']]

# Führe den Merge basierend auf den Titeln durch
merged_df = pd.merge(merged_df, netflix_df, on='title', how='left', suffixes=('', '_netflix'))

# Fülle die leeren Zellen in 'score' und 'votes' mit den Werten aus 'score_netflix' und 'votes_netflix'
merged_df['score'].fillna(merged_df['score_netflix'], inplace=True)
merged_df['votes'].fillna(merged_df['votes_netflix'], inplace=True)

# Entferne die zusätzlichen Spalten
merged_df.drop(columns=['score_netflix', 'votes_netflix'], inplace=True)

# Gib das kombinierte DataFrame aus
print(merged_df.head())

# Speichere das kombinierte DataFrame in einer neuen CSV-Datei
merged_df.to_csv('../../1_Datenset/erstellte/3_merged_with_votes.csv', index=False)

# Berechne die Anzahl der fehlenden Werte in den Spalten 'vote_count' und 'vote_average'
missing_vote_count_2 = merged_df['votes'].isnull().sum()
missing_vote_average_2 = merged_df['score'].isnull().sum()

print(f"Anzahl der fehlenden Werte in 'vote_count': {missing_vote_count_2}")
print(f"Anzahl der fehlenden Werte in 'vote_average': {missing_vote_average_2}")

# Berechne die Anzahl der vorhandenen Werte in den Spalten 'vote_count' und 'vote_average'
existing_vote_count_2 = merged_df['votes'].notnull().sum()
existing_vote_average_2 = merged_df['score'].notnull().sum()

print(f"Anzahl der vorhandenen Werte in 'vote_count': {existing_vote_count_2}")
print(f"Anzahl der vorhandenen Werte in 'vote_average': {existing_vote_average_2}")

  show_id   type                                             title  \
0      s1  Movie                              Dick Johnson Is Dead   
1      s1  Movie  Duck the Halls: A Mickey Mouse Christmas Special   
2     s10  Movie             A Muppets Christmas: Letters To Santa   
3     s10  Movie                                      The Starling   
4    s100  Movie                       Confessions of a Shopaholic   

                            director  \
0                    Kirsten Johnson   
1  Alonso Ramirez Ramos, Dave Wasson   
2                   Kirk R. Thatcher   
3                     Theodore Melfi   
4                         P.J. Hogan   

                                                cast        country  \
0                                            unknown  United States   
1  Chris Diamantopoulos, Tony Anselmo, Tress MacN...        unknown   
2  Steve Whitmire, Dave Goelz, Bill Barretta, Eri...  United States   
3  Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...  

/var/folders/q4/y1pgtglx2tl3lyq5msp5vs8m0000gp/T/ipykernel_37276/2249274819.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['score'].fillna(merged_df['score_netflix'], inplace=True)
/var/folders/q4/y1pgtglx2tl3lyq5msp5vs8m0000gp/T/ipykernel_37276/2249274819.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we ar

# 5. imbd movies noch dazu mergen


In [89]:
# Pfade zu den CSV-Dateien
imbd_file = '../../1_Datenset/ursprüngliche/imdb-movies-dataset.csv'
merged_file = '../../1_Datenset/erstellte/3_merged_with_votes.csv'

# Lade die Netflix CSV-Datei
imbd_df = pd.read_csv(imbd_file)

# Lade die merged CSV-Datei
merged_df = pd.read_csv(merged_file)

# Wähle nur die relevanten Spalten aus
imbd_df = imbd_df[['title', 'score', 'votes']]

# Führe den Merge basierend auf den Titeln durch
merged_df = pd.merge(merged_df, imbd_df, on='title', how='left', suffixes=('', '_imbd'))

# Fülle die leeren Zellen in 'score' und 'votes' mit den Werten aus 'score_netflix' und 'votes_netflix'
merged_df['score'].fillna(merged_df['score_imbd'], inplace=True)
merged_df['votes'].fillna(merged_df['votes_imbd'], inplace=True)

# Entferne die zusätzlichen Spalten
merged_df.drop(columns=['score_imbd', 'votes_imbd'], inplace=True)

# Gib das kombinierte DataFrame aus
print(merged_df.head())

# Speichere das kombinierte DataFrame in einer neuen CSV-Datei
merged_df.to_csv('../../1_Datenset/erstellte/4_merged_with_votes.csv', index=False)

# Berechne die Anzahl der fehlenden Werte in den Spalten 'vote_count' und 'vote_average'
missing_vote_count_3 = merged_df['votes'].isnull().sum()
missing_vote_average_3 = merged_df['score'].isnull().sum()

print(f"Anzahl der fehlenden Werte in 'vote_count': {missing_vote_count_3}")
print(f"Anzahl der fehlenden Werte in 'vote_average': {missing_vote_average_3}")

# Berechne die Anzahl der vorhandenen Werte in den Spalten 'vote_count' und 'vote_average'
existing_vote_count_3 = merged_df['votes'].notnull().sum()
existing_vote_average_3 = merged_df['score'].notnull().sum()

print(f"Anzahl der vorhandenen Werte in 'vote_count': {existing_vote_count_3}")
print(f"Anzahl der vorhandenen Werte in 'vote_average': {existing_vote_average_3}")

  show_id   type                                             title  \
0      s1  Movie                              Dick Johnson Is Dead   
1      s1  Movie  Duck the Halls: A Mickey Mouse Christmas Special   
2     s10  Movie             A Muppets Christmas: Letters To Santa   
3     s10  Movie                                      The Starling   
4    s100  Movie                       Confessions of a Shopaholic   

                            director  \
0                    Kirsten Johnson   
1  Alonso Ramirez Ramos, Dave Wasson   
2                   Kirk R. Thatcher   
3                     Theodore Melfi   
4                         P.J. Hogan   

                                                cast        country  \
0                                            unknown  United States   
1  Chris Diamantopoulos, Tony Anselmo, Tress MacN...        unknown   
2  Steve Whitmire, Dave Goelz, Bill Barretta, Eri...  United States   
3  Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...  

/var/folders/q4/y1pgtglx2tl3lyq5msp5vs8m0000gp/T/ipykernel_37276/2209792730.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['score'].fillna(merged_df['score_imbd'], inplace=True)
/var/folders/q4/y1pgtglx2tl3lyq5msp5vs8m0000gp/T/ipykernel_37276/2209792730.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are s

# 6. mymoviedb dazu

In [90]:
# Pfade zu den CSV-Dateien
mymovie_file = '../../1_Datenset/ursprüngliche/mymoviedb.csv'
merged_file = '../../1_Datenset/erstellte/3_merged_with_votes.csv'

# Lade die Netflix CSV-Datei
mymovie_df = pd.read_csv(mymovie_file)

# Lade die merged CSV-Datei
merged_df = pd.read_csv(merged_file)

# Wähle nur die relevanten Spalten aus
mymovie_df = mymovie_df[['title', 'score', 'votes']]

# Führe den Merge basierend auf den Titeln durch
merged_df = pd.merge(merged_df, mymovie_df, on='title', how='left', suffixes=('', '_imbd'))

# Fülle die leeren Zellen in 'score' und 'votes' mit den Werten aus 'score_netflix' und 'votes_netflix'
merged_df['score'].fillna(merged_df['score_imbd'], inplace=True)
merged_df['votes'].fillna(merged_df['votes_imbd'], inplace=True)

# Entferne die zusätzlichen Spalten
merged_df.drop(columns=['score_imbd', 'votes_imbd'], inplace=True)

# Gib das kombinierte DataFrame aus
print(merged_df.head())

# Speichere das kombinierte DataFrame in einer neuen CSV-Datei
merged_df.to_csv('../../1_Datenset/erstellte/5_merged_with_votes.csv', index=False)

# Berechne die Anzahl der fehlenden Werte in den Spalten 'vote_count' und 'vote_average'
missing_vote_count_4 = merged_df['votes'].isnull().sum()
missing_vote_average_4 = merged_df['score'].isnull().sum()

print(f"Anzahl der fehlenden Werte in 'vote_count': {missing_vote_count_4}")
print(f"Anzahl der fehlenden Werte in 'vote_average': {missing_vote_average_4}")

# Berechne die Anzahl der vorhandenen Werte in den Spalten 'vote_count' und 'vote_average'
existing_vote_count_4 = merged_df['votes'].notnull().sum()
existing_vote_average_4 = merged_df['score'].notnull().sum()

print(f"Anzahl der vorhandenen Werte in 'vote_count': {existing_vote_count_4}")
print(f"Anzahl der vorhandenen Werte in 'vote_average': {existing_vote_average_4}")

  show_id   type                                             title  \
0      s1  Movie                              Dick Johnson Is Dead   
1      s1  Movie  Duck the Halls: A Mickey Mouse Christmas Special   
2     s10  Movie             A Muppets Christmas: Letters To Santa   
3     s10  Movie                                      The Starling   
4    s100  Movie                       Confessions of a Shopaholic   

                            director  \
0                    Kirsten Johnson   
1  Alonso Ramirez Ramos, Dave Wasson   
2                   Kirk R. Thatcher   
3                     Theodore Melfi   
4                         P.J. Hogan   

                                                cast        country  \
0                                            unknown  United States   
1  Chris Diamantopoulos, Tony Anselmo, Tress MacN...        unknown   
2  Steve Whitmire, Dave Goelz, Bill Barretta, Eri...  United States   
3  Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...  

/var/folders/q4/y1pgtglx2tl3lyq5msp5vs8m0000gp/T/ipykernel_37276/1005945484.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['score'].fillna(merged_df['score_imbd'], inplace=True)
/var/folders/q4/y1pgtglx2tl3lyq5msp5vs8m0000gp/T/ipykernel_37276/1005945484.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... '7.1' '7.3' nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  